<a href="https://colab.research.google.com/github/dewdevil99/Image-Captioning-System/blob/main/Image_Captioning_Final_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import string
import numpy as np
from PIL import Image
import os
import random
from pickle import dump, load
import pandas as pd
import nltk
import matplotlib.pyplot as plt
%matplotlib inline
from nltk.corpus import stopwords
nltk.download('stopwords')
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers.merge import add
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, Embedding, Dropout

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Instructions for importing the dataset from Google Drive

Use pd.read_csv('/content/gdrive/My Drive/AI Project Dataset/Flickr8k_text/file_name_as_per_data_flair')

Also, /content/gdrive/My Drive/AI Project Dataset/Flickr8k_Dataset contains images.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#loading image names and captions from .txt files (from our dataset) and 
#creating a dictionary consisting of image names as keys and list of captions as values
#after that the captions in the description are cleaned (removing '-', punctuation marks, converting to lower case, removing numbers, etc.)

# Loading a text file into memory
def load_doc(filename):
    # Opening the file as read only
    file = open(filename, 'r')
    text = file.read()
    file.close()
    return text

# get all imgs with their captions
def all_img_captions(filename):
    file = load_doc(filename)
    captions = file.split('\n')
    descriptions ={}
    for caption in captions[:-1]:
        img, caption = caption.split('\t')
        if img[:-2] not in descriptions:
            descriptions[img[:-2]] = [ caption ]
        else:
            descriptions[img[:-2]].append(caption)
    return descriptions

#Data cleaning- lower casing, removing puntuations and words containing numbers
def cleaning_text(captions):
    table = str.maketrans('','',string.punctuation)
    for img,caps in captions.items():
        for i,img_caption in enumerate(caps):

            img_caption.replace("-"," ")
            desc = img_caption.split()

            #converts to lowercase
            desc = [word.lower() for word in desc]
            #remove punctuation from each token
            desc = [word.translate(table) for word in desc]
            #remove hanging 's and a
            desc = [word for word in desc if(len(word)>1)]
            #remove tokens with numbers in them
            desc = [word for word in desc if(word.isalpha())]
            desc1 = []
            for word in desc:
                x = random.random()
                if word == 'man':
                    if x <= 0.2:
                        desc1.append('person')
                    else:
                        desc1.append(word)
                elif word == 'dog':
                    if x <= 0.25:
                        desc1.append('pet-dog')
                    else:
                        desc1.append(word)
                elif word == 'two':
                    if x <= 0.18:
                        desc1.append('couple of')
                    else:
                        desc1.append(word)
                else:
                    desc1.append(word)
            desc = desc1
            #convert back to string

            img_caption = ' '.join(desc)
            captions[img][i]= img_caption
    return captions

def text_vocabulary(descriptions):
    # build vocabulary of all unique words
    vocab = set()

    for key in descriptions.keys():
        [vocab.update(d.split()) for d in descriptions[key]]

    return vocab

#All descriptions in one file 
def save_descriptions(descriptions, filename):
    lines = list()
    for key, desc_list in descriptions.items():
        for desc in desc_list:
            lines.append(key + '\t' + desc )
    data = "\n".join(lines)
    file = open(filename,"w")
    file.write(data)
    file.close()


# Set these path according to project folder in you system
dataset_text = "/content/drive/MyDrive/AI Project Dataset/Flickr8k_text"
dataset_images = "/content/drive/MyDrive/AI Project Dataset/Flickr8k_Dataset"

#we prepare our text data
filename = dataset_text + "/" + "Flickr8k.token.txt"
#loading the file that contains all data
#mapping them into descriptions dictionary img to 5 captions
descriptions = all_img_captions(filename)
print("Length of descriptions =" ,len(descriptions))

#cleaning the descriptions
clean_descriptions = cleaning_text(descriptions)

#building vocabulary 
vocabulary = text_vocabulary(clean_descriptions)
print("Length of vocabulary = ", len(vocabulary))

#saving each description to file 
save_descriptions(clean_descriptions, "/content/drive/MyDrive/AI dummy folder/descriptions.txt")

In [ ]:
#paths to the different files of the dataset
dataset_text = "/content/drive/MyDrive/AI Project Dataset/Flickr8k_text"
dataset_images = "/content/drive/MyDrive/AI Project Dataset/Flickr8k_Dataset"

In [ ]:
#in this cell, we convert all the images in the entire dataset into feature vectors
#by doing a forward pass through the Xception network for all the images
#these vectors are saved as a pickle file features.p
#as this code segment takes a lot of time to run, we run it only once
#everytime we want to use feature vectors, we load them through features.p
def extract_features(directory):
    model = Xception( include_top=False, pooling='avg' )
    features = {}
    i=0
    for img in os.listdir(directory):
        i += 1
        print(i) 
        filename = directory + "/" + img
        image = Image.open(filename)
        image = image.resize((299,299))
        image = np.expand_dims(image, axis=0)
        #image = preprocess_input(image)
        image = image/127.5
        image = image - 1.0

        feature = model.predict(image)
        # print(type(feature))
        # print("length",len(feature))
        # print(feature)
        # print(feature.shape)
        features[img] = feature
    return features
print("Done")
#2048 feature vector
features = extract_features(dataset_images)
dump(features, open("features.p","wb"))

In [ ]:
#feature vectors loaded through the pickle file
features = load(open("/content/drive/MyDrive/AI Project Dataset/features.p","rb"))

In [ ]:
#here we load the names of the training images from the dataset
#and with the help of that, we create a dictionary consisting of training image names as keys and list of captions as values
#in these captions, we have added <start> at the beginning and <end> at the end
#after that we create a dictionary of training images and their feature vectors named train_features

#load the data 
def load_photos(filename):
    file = load_doc(filename)
    photos = file.split("\n")[:-1]
    return photos


def load_clean_descriptions(filename, photos): 
    #loading clean_descriptions
    file = load_doc(filename)
    descriptions = {}
    for line in file.split("\n"):

        words = line.split()
        if len(words)<1 :
            continue

        image, image_caption = words[0], words[1:]

        if image in photos:
            if image not in descriptions:
                descriptions[image] = []
            desc = '<start> ' + " ".join(image_caption) + ' <end>'
            descriptions[image].append(desc)

    return descriptions


def load_features(photos):
    #loading all features
    all_features = load(open("/content/drive/MyDrive/AI Project Dataset/features.p","rb"))
    #selecting only needed features
    features = {k:all_features[k] for k in photos}
    return features


filename = dataset_text + "/" + "Flickr_8k.trainImages.txt"

#train = loading_data(filename)
train_imgs = load_photos(filename) #list containing training image IDs as strings
train_descriptions = load_clean_descriptions("/content/drive/MyDrive/AI dummy folder/descriptions.txt", train_imgs)
train_features = load_features(train_imgs)

In [ ]:
#here we create a tokenizer to vectorise the text corpus (captions), that is,
#it converts all the words in the training captions into integers based on their frequency
#this is done because we can feed only numbers to our NN

#converting dictionary to clean list of descriptions
def dict_to_list(descriptions):
    all_desc = []
    for key in descriptions.keys():
        [all_desc.append(d) for d in descriptions[key]]
    return all_desc

#creating tokenizer class 
#this will vectorise text corpus
#each integer will represent token in dictionary

from keras.preprocessing.text import Tokenizer

def create_tokenizer(descriptions):
    desc_list = dict_to_list(descriptions)
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(desc_list)
    return tokenizer

# give each word an index, and store that into tokenizer.p pickle file
tokenizer = create_tokenizer(train_descriptions)
dump(tokenizer, open('tokenizer.p', 'wb'))
dump(tokenizer, open("/content/drive/MyDrive/AI dummy folder/tokenizer.p","wb"))
vocab_size = len(tokenizer.word_index) + 1
vocab_size

In [ ]:
#here we calculate the maximum length of the captions


#calculate maximum length of descriptions
def max_length(descriptions):
    desc_list = dict_to_list(descriptions)
    return max(len(d.split()) for d in desc_list)
    
max_length = max_length(descriptions)
max_length

In [ ]:
#data visualization: stop words are removed here

tokenizer = load(open("/content/drive/MyDrive/AI dummy folder/tokenizer.p","rb"))


def order_by_second_arg(my_tuple):
    return my_tuple[1]

train_all_words = list(tokenizer.word_counts.keys())
train_word_counts = list(tokenizer.word_counts.values())
words_and_counts = list(zip(train_all_words, train_word_counts))
words_and_counts.sort(reverse=True, key=order_by_second_arg)
print(words_and_counts)


def remove_stop_words(l, stop_words):
    l = [(a,b) for (a,b) in l if a not in stop_words]
    return l

def unzip(l):
    l1 = []
    l2 = []
    for a,b in l:
        l1.append(a)
        l2.append(b)
    return (l1,l2)

stop_words = set(stopwords.words('english'))
stop_words = list(stop_words)

words_and_counts = remove_stop_words(words_and_counts, stop_words)
train_all_words, train_word_counts = unzip(words_and_counts)
#"'s" in train_all_words


fig = plt.figure()
axes = fig.add_axes([0.1, 0.1, 2.8, 2.8]) # left, bottom, width, height (range 0 to 1)
axes.plot(train_all_words[2:20], train_word_counts[2:20], 'b')
axes.set_xlabel('Words')
axes.set_ylabel('Frequency')
axes.set_title('Frequency of words occuring in the descriptions of images in the training set')

In [ ]:
#here we create a data generator to generate data one after the other
#so that it can be fed into the model during training, as so much data cannot be loaded into the memory at once
#here the data generated is of the form: 
#inputs are partial sentences and the feature vector of the image, outputs are the correct/expected next word
#in this way, we teach the model to predict the next word, given a partial sentence


# data generator, intended to be used in a call to model.fit_generator()
def data_generator(descriptions, photos, tokenizer, max_length, vocab_size):
	# loop for ever over images
	while 1:
		for key, desc_list in descriptions.items():
			# retrieve the photo feature
			photo = photos[key][0]
			in_img, in_seq, out_word = create_sequences(tokenizer, max_length, desc_list, photo, vocab_size)
			yield ((in_img, in_seq), out_word)

# create sequences of images, input sequences and output words for an image
def create_sequences(tokenizer, max_length, desc_list, photo, vocab_size):
	X1, X2, y = list(), list(), list()
	# walk through each description for the image
	for desc in desc_list:
		# encode the sequence
		seq = tokenizer.texts_to_sequences([desc])[0]
		# split one sequence into multiple X,y pairs
		for i in range(1, len(seq)):
			# split into input and output pair
			in_seq, out_seq = seq[:i], seq[i]
			# pad input sequence
			in_seq = pad_sequences([in_seq], maxlen=max_length)[0]
			# encode output sequence
			out_seq = to_categorical([out_seq], num_classes=vocab_size)[0]
			# store
			X1.append(photo)
			X2.append(in_seq)
			y.append(out_seq)
	return np.array(X1), np.array(X2), np.array(y)

In [ ]:
#Here we define our model. We have a feature vector 2048 input layer followed by a Dense layer.
#Then we have an Input layer of the length same as the max length of the captions. Now we combine these two
#layers and complete our Deep NN model. The final layer is Dense layer with vocabulary size many classes

from keras.utils import plot_model

# define the captioning model
def define_model(vocab_size, max_length):

    # features from the CNN model squeezed from 2048 to 256 nodes
    inputs1 = Input(shape=(2048,))
    #fe1 = Dropout(0.05)(inputs1)
    fe2 = Dense(256, activation='relu')(inputs1)


    # LSTM sequence model
    inputs2 = Input(shape=(max_length,))
    se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
    #se2 = Dropout(0.25)(se1)
    se3 = LSTM(256)(se1)

    # Merging both models
    decoder1 = add([fe2, se3])
    
    
    #decoder4 = Dropout(0.15)(decoder3)
    outputs = Dense(vocab_size, activation='softmax')(decoder1)

    # tie it together [image, seq] [word]
    model = Model(inputs=[inputs1, inputs2], outputs=outputs)
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    # summarize model
    print(model.summary())
    plot_model(model, to_file='model.png', show_shapes=True)

    return model

In [ ]:
#Here we train the model with 30 epochs. It was observed that the model converges to the final accuracy by 
#epoch 26

# train the model, run epochs manually and save after each epoch
epochs = 30
model = define_model(vocab_size, max_length)
steps = len(train_descriptions)
#os.mkdir("models")

for i in range(epochs):
  # create the data generator
  generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
  # fit for one epoch
  model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)
# save model
  #model.save("models/model_8k_3_" + str(i) + ".h5")
  model.save("/content/drive/MyDrive/AI dummy folder/model_8k_3_" + str(i) + ".h5")

In [ ]:
#This code is to resume training if the traning of the model stopped due to some unforseen (Google-Colab-runtime related) reason

epochs=18
steps = len(train_descriptions)
#os.mkdir("models")
#model = define_model(vocab_size, max_length)
model = load_model("/content/drive/MyDrive/AI dummy folder/model_8k_310.h5")#"models/model_9.h5"
for i in range(epochs):
  # create the data generator
  generator = data_generator(train_descriptions, train_features, tokenizer, max_length, vocab_size)
  # fit for one epoch
  model.fit_generator(generator, epochs=1, steps_per_epoch=steps, verbose=1)
  # save model
  #model.save("models/model_8k_2" + str(i+2) + ".h5")
  model.save("/content/drive/MyDrive/AI dummy folder/model_8k_3" + str(i+11) + ".h5")

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import argparse
import random

#We extract the 2048 fature vector in this fucntion
def extract_features(filename, model):
        try:
            image = Image.open(filename)

        except:
            print("ERROR: Couldn't open image! Make sure the image path and extension is correct")
        image = image.resize((299,299))
        image = np.array(image)
        # for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

#we get the word corresponding to a particular index from here
def word_for_id(integer, tokenizer):
  for word,index in tokenizer.word_index.items():
    if index == integer:
         return word
  return None

#We generate the description here for a given model
def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text



max_length = 38
tokenizer = load(open("tokenizer.p","rb"))
model = load_model("/content/drive/MyDrive/AI dummy folder/model_8k_326.h5")
xception_model = Xception(include_top=False, pooling="avg")
i=0
#start = random.randint(1,2000)
start = 1145
print(start)
for im in os.listdir(dataset_images + "/"):
  if im not in train_imgs:
    if i >=start:
      photo = extract_features(dataset_images + "/" + im, xception_model)
      img = Image.open(dataset_images + "/" + im)
      captions = descriptions[im]

      description = generate_desc(model, tokenizer, photo, max_length)
      hypo = description.split()
      ref = []
      for c in captions:
        caption = c.split()
        ref.append(caption)

      bleuScore_bi = nltk.translate.bleu_score.sentence_bleu(ref,hypo,weights=(0,1,0,0))
      bleuScore_uni = nltk.translate.bleu_score.sentence_bleu(ref,hypo,weights=(1,0,0,0))
      bleuScore_cum = nltk.translate.bleu_score.sentence_bleu(ref,hypo,weights=(0.25,0.25,0.25,0.25))
      print("\n\n")
      print(description)
      print("\n")
      print("BLEU Score Uni = ",bleuScore_uni)
      print("BLEU Score Bi = ",bleuScore_bi)
      print("BLEU Score Cum= ",bleuScore_cum)
      plt.imshow(img)
      plt.pause(5)
    if i>start + 10:
      break
    i+=1

In [ ]:
#Code to predict captions on a video

from google.colab.patches import cv2_imshow
import cv2
import time

def extract_features(frame, model):
        image=cv2.resize(frame,(299,299),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        image = np.array(image)
        #for images that has 4 channels, we convert them into 3 channels
        if image.shape[2] == 4: 
            image = image[..., :3]
        image = np.expand_dims(image, axis=0)
        image = image/127.5
        image = image - 1.0
        feature = model.predict(image)
        return feature

def word_for_id(integer, tokenizer):
  for word,index in tokenizer.word_index.items():
    if index == integer:
         return word
  return None


def generate_desc(model, tokenizer, photo, max_length):
    in_text = 'start'
    for i in range(max_length):
        sequence = tokenizer.texts_to_sequences([in_text])[0]
        sequence = pad_sequences([sequence], maxlen=max_length)
        pred = model.predict([photo,sequence], verbose=0)
        pred = np.argmax(pred)
        word = word_for_id(pred, tokenizer)
        if word is None:
            break
        in_text += ' ' + word
        if word == 'end':
            break
    return in_text

# Same command function as streaming, its just now we pass in the file path, nice!
cap = cv2.VideoCapture('/content/drive/MyDrive/AI dummy folder/video.mov')


# Always a good idea to check if the video was acutally there
# If you get an error at thsi step, triple check your file path!!
if cap.isOpened()== False: 
    print("Error opening the video file. Please double check your file path for typos. Or move the movie file to the same location as this script/notebook")
    
frame_seq=0
time_length = 11
fps=24
# While the video is opened
while cap.isOpened():
    
    
    #print("Inside while")
    frame_no = (frame_seq /(time_length*fps))
    cap.set(1,frame_seq)
    # Read the video file.
    ret, frame = cap.read()
    
    # If we got frames, show them.
    if ret == True:
        photo = extract_features(frame, xception_model)
        
        description = generate_desc(model, tokenizer, photo, max_length)
        print("\n\n")
        print(description)
        frame=cv2.resize(frame,(450,300),fx=0,fy=0, interpolation = cv2.INTER_CUBIC)
        cv2_imshow(frame)
        #plt.imshow(frame)
        frame_seq+=48

        # Press q to quit
        if cv2.waitKey(25) & 0xFF == ord('q'):
            
            break
        plt.pause(5) 
    # Or automatically break this whole loop if the video is over.
    else:
        print("Breaking loop")
        break
        
cap.release()
# Closes all the frames
cv2.destroyAllWindows()